<a href="https://colab.research.google.com/github/keerthucit/EIP2Assignment/blob/master/Projwel4a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.initializers import VarianceScaling
from keras.regularizers import l2

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64 # or 128
num_classes = 10
epochs = 100 # increase to 250
depth = 100 # total num layers - 40 for	DN, 100 for DN BC
k = 12 # growth rate
compression = 0.5 # reduction
dropout_rate = None # None for augmented  # 0.2 for non augmented

learning_rate = 0.1 # initial
weight_decay = 1e-4 # DCNNN paper
momentum = 0.9 # Nesterov

num_blocks = 3 # number of dense blocks in the network

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# convert to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize - TODO check channel mean and std normalization later
#x_train /= 255
#x_test /= 255
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
# save for later use in visualization
y_train_orig = y_train
y_test_orig = y_test
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

170500096/170498071 [==============================] - 13s 0us/step


In [0]:
from keras.preprocessing.image import ImageDataGenerator

# Normalization - channelwise 0 mean and 1 std

train_gen = ImageDataGenerator( featurewise_center=True,
                            featurewise_std_normalization=True,
                            #rescale=1./255,
                            width_shift_range=2./32,
                            height_shift_range=2./32,
                            horizontal_flip=True)

train_gen.fit(x_train, seed=0)
print (train_gen.mean)
print (train_gen.std)

[[[125.3069  122.95015 113.866  ]]]
[[[62.993256 62.08861  66.705   ]]]


In [0]:
train_datagen = train_gen.flow(x_train, y_train, batch_size=1)

In [0]:
print(x_train[0,:,:,0])

[[ 59.  43.  50. ... 158. 152. 148.]
 [ 16.   0.  18. ... 123. 119. 122.]
 [ 25.  16.  49. ... 118. 120. 109.]
 ...
 [208. 201. 198. ... 160.  56.  53.]
 [180. 173. 186. ... 184.  97.  83.]
 [177. 168. 179. ... 216. 151. 123.]]


In [0]:
x,y = train_datagen.next()
print(x[0,:,:,0])

[[-0.92228484 -0.99321806 -0.9427898  ... -0.6203346  -0.65205014
  -0.66783726]
 [-0.786311   -0.8256106  -0.7782492  ... -0.63386333 -0.65530664
  -0.6950303 ]
 [-0.43443322 -0.4506387  -0.4517148  ... -0.62732035 -0.6605105
  -0.7388816 ]
 ...
 [-0.28876945 -0.04232284  0.42621127 ... -1.002245   -0.5826227
  -0.30122414]
 [ 0.10015173  0.45103657  0.7858372  ... -0.95673853 -0.41297936
  -0.26869112]
 [ 0.38309643  0.6771028   0.86378235 ... -0.90276605 -0.33189324
  -0.2747421 ]]


In [0]:
# normalize the test set with train set params

test_gen = ImageDataGenerator(featurewise_center=True,
                            featurewise_std_normalization=True)

test_gen.fit(x_train, seed=0)
print(test_gen.mean)
print(test_gen.std)


[[[125.3069  122.95015 113.866  ]]]
[[[62.993256 62.08861  66.705   ]]]


In [0]:
test_datagen = test_gen.flow(x_test, y_test, batch_size=1)

In [0]:
print(x_test[0,:,:,0])

[[158. 159. 165. ... 137. 126. 116.]
 [152. 151. 159. ... 136. 125. 119.]
 [151. 151. 158. ... 139. 130. 120.]
 ...
 [ 68.  42.  31. ...  38.  13.  40.]
 [ 61.  49.  35. ...  26.  29.  20.]
 [ 54.  56.  45. ...  24.  34.  21.]]


In [0]:
x,y = test_datagen.next()
print(x[0,:,:,0])

[[-0.0524961  -0.10012024 -0.11599496 ... -0.06837081 -0.08424553
  -0.11599496]
 [-0.10012024 -0.14774439 -0.1636191  ... -0.13186967 -0.13186967
  -0.14774439]
 [-0.11599496 -0.1636191  -0.1636191  ... -0.1636191  -0.17949381
  -0.21124326]
 ...
 [ 0.9793604   1.3286041   1.5825996  ...  1.2333559   1.2492305
   1.28098   ]
 [ 1.1063582   1.4873513   1.6460985  ...  1.28098     1.2651052
   1.2651052 ]
 [ 1.2174811   1.5667249   1.6778479  ...  1.3603536   1.2968547
   1.2651052 ]]


In [0]:
import numpy as np
mn  = np.mean(x_train, (0,1,2))
print (mn)
sd = np.std(x_train, (0,1,2))
print (sd)
sd = np.std(x_train)
print (sd)

[125.3069  122.95015 113.866  ]
[62.99325  62.088604 66.70501 ]
64.150024


In [0]:
print (np.mean(x_test, (0,1,2)))
print (np.std(x_test, (0,1,2)))
print (np.std(x_test))

[126.02428 123.70843 114.85442]
[62.896416 61.937508 66.70607 ]
64.06093


In [0]:
# normalize test data with standardize function of generator

#x_test_orig = x_test.copy()
#for i in range(len(x_test)):
#  x_test[i]=test_gen.standardize(x_test[i])

In [0]:
#print (np.mean(x_test, (0,1,2)))
#print (np.std(x_test, (0,1,2)))
#print (np.std(x_test))

[-1.9813678 -1.9724256 -1.7002549]
[0.00391554 0.00391203 0.00392164]
0.13051839


In [0]:
## normalize with sklearn scaler
#from sklearn import preprocessing

#scaler = preprocessing.StandardScaler().fit(x_train)
#x_test_norm = scaler.transform(x_test)

In [0]:
# Dense Block
def add_denseblock(input, num_layers, num_filter, growth_rate, dropout_rate = None):
    global compression
    global init_mrsa
    global weight_decay
    temp = input
    for _ in range(num_layers):
        BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay/2),
                                       beta_regularizer=l2(weight_decay/2))(temp)
        relu = Activation('relu')(BatchNorm)
        # 1x1 conv - bottleneck layer
        Conv2D_1_1 = Conv2D(int(4*growth_rate),
            (1,1),
            use_bias=False,
            padding='same',
            kernel_initializer=init_mrsa,
            kernel_regularizer=l2(weight_decay/2))(relu)
        # 3x3 conv
        Conv2D_3_3 = Conv2D(int(growth_rate),
            (3,3), 
            use_bias=False, 
            padding='same',
            kernel_initializer=init_mrsa,
            kernel_regularizer=l2(weight_decay/2))(Conv2D_1_1)
        if dropout_rate is not None:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        # concat outputs
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        num_filter += growth_rate
        
        temp = concat
        
    return temp, num_filter


In [0]:
def add_transition(input, num_filter, dropout_rate = None):
    global compression
    global init_mrsa
    global weight_decay
    BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay/2), 
                                beta_regularizer=l2(weight_decay/2))(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), 
        kernel_initializer=init_mrsa,
        use_bias=False, 
        padding='same',
        kernel_regularizer=l2(weight_decay/2))(relu)
    if dropout_rate is not None:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2), strides=(2,2))(Conv2D_BottleNeck)
    
    return avg, int(num_filter*compression)

In [0]:
def output_layer(input, size):
    global compression
    global weight_decay
    BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay/2), 
                                beta_regularizer=l2(weight_decay/2))(input)
    relu = Activation('relu')(BatchNorm)
    #AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    # try global lavg pooling
    AvgPooling = AveragePooling2D(pool_size=(size,size))(relu)
    # try 2x2 with 2x2 stride - gives 4x4x27
    #AvgPooling = AveragePooling2D(pool_size=(2,2), strides=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

 

In [0]:
# num layers in each block
num_layers = int((depth - 4)/num_blocks) 

input = Input(shape=(img_height, img_width, channel,))
init_mrsa = VarianceScaling(2.0) # mode = FAN_IN, distribution=normal, to get MRSA init
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/variance_scaling_initializer
# https://medium.com/intuitionmachine/notes-on-the-implementation-densenet-in-tensorflow-beeda9dd1504

# initial number of filters - output of 1st layer is 2 * growth rate
num_filter = k * 2 

output = Conv2D(num_filter,  
                (3,3), 
                kernel_initializer=init_mrsa,
                use_bias=False,
                padding='same',
                kernel_regularizer=l2(weight_decay/2))(input) # https://bbabenko.github.io/weight-decay/
#32x32, 24

print (f'num_layers {num_layers}')
for block in range(num_blocks): 
    # num_layers/2 due to bottleneck net - 1 1x1 and 1 3x3 in each step
    print (f'DN: block {block} DB : in num_filter {num_filter}')
    output, num_filter = add_denseblock(output, int(num_layers/2), num_filter, k, dropout_rate)
    print (f'DN: block {block} DB : out num_filter {num_filter}')
#32x32, 24+12=36, 16x16x18+12=30, 8x8x15+12=27
    if (block != (num_blocks-1)):
        print (f'DN: block {block} TR : in num_filter {num_filter}')
        output, num_filter = add_transition(output, num_filter, dropout_rate)
        print (f'DN: block {block} TR : out num_filter {num_filter}')
#32x32, 16x16x36/2=18, 8x8x30/2=15, 

# 8x8x27
output = output_layer(output, 8)


num_layers 32
DN: block 0 DB : in num_filter 24
DN: block 0 DB : out num_filter 216
DN: block 0 TR : in num_filter 216
DN: block 0 TR : out num_filter 108
DN: block 1 DB : in num_filter 108
DN: block 1 DB : out num_filter 300
DN: block 1 TR : in num_filter 300
DN: block 1 TR : out num_filter 150
DN: block 2 DB : in num_filter 150
DN: block 2 DB : out num_filter 342


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
from keras.optimizers import SGD
# LR decay = 0.1/250/4 = 1.0e-4
sgd = SGD(lr=learning_rate,momentum=momentum, decay=1.0e-4, nesterov=True)

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
#              optimizer=Adam(),
              optimizer=sgd,
              metrics=['accuracy'])


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

#filepath="/content/gdrive/My Drive/EIP2/Session4/1/DCNN_v2_best.h5"
filepath="/content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-{epoch:02d}-{val_acc:.2f}.h5"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
with open('/content/gdrive/My Drive/EIP2/Session4/2/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/EIP2/Session4/1/foo.txt

Hello Google Drive!

In [0]:
!rm -rf clr_callback.py*
!wget https://github.com/bckenstler/CLR/raw/master/clr_callback.py
from clr_callback import *

--2018-11-03 18:23:55--  https://github.com/bckenstler/CLR/raw/master/clr_callback.py
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py [following]
--2018-11-03 18:23:56--  https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5326 (5.2K) [text/plain]
Saving to: ‘clr_callback.py’

clr_callback.py     100%[===================>]   5.20K  --.-KB/s    in 0s      

2018-11-03 18:23:57 (67.8 MB/s) - ‘clr_callback.py’ saved [5326/5326]



In [0]:
from keras.callbacks import ModelCheckpoint

#filepath="DNST_model-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [0]:
# add CLR callback
clr_triangular = CyclicLR(mode='triangular2', base_lr=0.0003, max_lr=0.1, step_size=2500)
callbacks_list.append(clr_triangular)

In [0]:
model.fit_generator(train_gen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch = x_train.shape[0]/(batch_size/2),
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_gen.flow(x_test, y_test, batch_size=50),
                    callbacks=callbacks_list)


Epoch 1/100
1563/1562 [==============================] - 626s 401ms/step - loss: 1.9488 - acc: 0.5552 - val_loss: 2.6957 - val_acc: 0.5299

Epoch 00001: val_acc improved from -inf to 0.52990, saving model to /content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-01-0.53.h5
Epoch 2/100
1563/1562 [==============================] - 614s 393ms/step - loss: 1.3390 - acc: 0.7574 - val_loss: 1.2676 - val_acc: 0.7739

Epoch 00002: val_acc improved from 0.52990 to 0.77390, saving model to /content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-02-0.77.h5
Epoch 3/100
1563/1562 [==============================] - 614s 393ms/step - loss: 1.0150 - acc: 0.8455 - val_loss: 0.9674 - val_acc: 0.8591

Epoch 00003: val_acc improved from 0.77390 to 0.85910, saving model to /content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-03-0.86.h5
Epoch 4/100
1563/1562 [==============================] - 614s 393ms/step - loss: 0.8744 - acc: 0.8877 - val_loss: 0.9980 - val_acc: 0.8487

Epoch 00004: val_acc did not improve from 0.85910


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

#filepath="/content/gdrive/My Drive/EIP2/Session4/1/DCNN_v2_best.h5"
filepath="/content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-2-{epoch:02d}-{val_acc:.2f}.h5"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
with open('/content/gdrive/My Drive/EIP2/Session4/2/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/EIP2/Session4/1/foo.txt

Hello Google Drive!

In [0]:
best_model = "DCNN_v3-20-0.90.h5"
model_path = "/content/gdrive/My Drive/EIP2/Session4/2/"+best_model
print (model_path)

/content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-20-0.90.h5


In [0]:
from keras.models import load_model
model = load_model(model_path)

In [0]:
# Test the model
#score = model.evaluate(x_test, y_test, verbose=1)
score = model.evaluate_generator(test_gen.flow(x_test, y_test, batch_size=1), verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 386s 39ms/step
Test loss: 0.7599838314890861
Test accuracy: 0.9029


In [0]:
from keras.optimizers import SGD
# LR decay = 0.1/250/4 = 1.0e-4
sgd = SGD(lr=learning_rate,momentum=momentum, decay=1.0e-4, nesterov=True)

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
#              optimizer=Adam(),
              optimizer=sgd,
              metrics=['accuracy'])


In [0]:
!rm -rf clr_callback.py*
!wget https://github.com/bckenstler/CLR/raw/master/clr_callback.py
from clr_callback import *

--2018-11-03 02:36:14--  https://github.com/bckenstler/CLR/raw/master/clr_callback.py
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py [following]
--2018-11-03 02:36:14--  https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5326 (5.2K) [text/plain]
Saving to: ‘clr_callback.py’

clr_callback.py     100%[===================>]   5.20K  --.-KB/s    in 0s      

2018-11-03 02:36:14 (62.4 MB/s) - ‘clr_callback.py’ saved [5326/5326]



In [0]:
from keras.callbacks import ModelCheckpoint

#filepath="DNST_model-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [0]:
# add CLR callback
clr_triangular = CyclicLR(mode='triangular2', base_lr=0.0003, max_lr=0.1, step_size=2500)
callbacks_list.append(clr_triangular)

In [0]:
model.fit_generator(train_gen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch = x_train.shape[0]/(batch_size/2),
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_gen.flow(x_test, y_test, batch_size=50),
                    callbacks=callbacks_list)


Epoch 1/100
1563/1562 [==============================] - 629s 402ms/step - loss: 0.7535 - acc: 0.8911 - val_loss: 1.1682 - val_acc: 0.7648

Epoch 00001: val_acc improved from -inf to 0.76480, saving model to /content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-2-01-0.76.h5
Epoch 2/100
1563/1562 [==============================] - 618s 395ms/step - loss: 0.8187 - acc: 0.8604 - val_loss: 0.8647 - val_acc: 0.8477

Epoch 00002: val_acc improved from 0.76480 to 0.84770, saving model to /content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-2-02-0.85.h5
Epoch 3/100
1563/1562 [==============================] - 618s 395ms/step - loss: 0.6480 - acc: 0.9088 - val_loss: 0.6702 - val_acc: 0.8990

Epoch 00003: val_acc improved from 0.84770 to 0.89900, saving model to /content/gdrive/My Drive/EIP2/Session4/2/DCNN_v3-2-03-0.90.h5
Epoch 4/100
1563/1562 [==============================] - 619s 396ms/step - loss: 0.5405 - acc: 0.9423 - val_loss: 0.7036 - val_acc: 0.8880

Epoch 00004: val_acc did not improve from 0.